In [1]:
import pandas as pd
from datetime import datetime, timedelta

base_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"

# Generar fechas de febrero 2020
dates = pd.date_range(start='2020-02-01', end='2020-02-29', freq='D')

dfs = []
for date in dates:
    try:
        # Formato: MM-DD-YYYY
        filename = date.strftime('%m-%d-%Y') + '.csv'
        url = base_url + filename
        print(f"Cargando: {filename}")
        
        df = pd.read_csv(url)
        df['Date'] = date  # Agregar columna de fecha
        dfs.append(df)
    except Exception as e:
        print(f"Error con {filename}: {e}")

# Concatenar todos los DataFrames
df_enero = pd.concat(dfs, ignore_index=True)
print(f"\nTotal de registros: {len(df_enero)}")
print(f"Fechas cargadas: {df_enero['Date'].min()} a {df_enero['Date'].max()}")

Cargando: 02-01-2020.csv
Error con 02-01-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-02-2020.csv
Error con 02-02-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-03-2020.csv
Error con 02-03-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-04-2020.csv
Error con 02-04-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-05-2020.csv
Error con 02-05-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-06-2020.csv
Error con 02-06-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-07-2020.csv
Error con 02-07-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-08-2020.csv
Error con 02-08-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-09-2020.csv
Cargando: 02-10-2020.csv
Error con 02-10-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-11-2020.csv
Error con 02-11-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-12-2020.csv
Error con 02-12-2020.csv: HTTP Error 429: Too Many Requests
Cargando: 02-13-2020.csv
Error con 02-13

In [7]:
df_enero.sample(5, random_state=42)


,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date
77,Kedah,Malaysia,2020-02-09T23:33:02,0,0,0,2020-02-09
10,Sichuan,Mainland China,2020-02-09T12:43:01,386,1,71,2020-02-09
4,Hunan,Mainland China,2020-02-09T12:33:01,838,1,186,2020-02-09
83,Perlis,Malaysia,2020-02-09T23:33:02,0,0,0,2020-02-09
62,NaN,Nepal,2020-01-31T08:15:53,1,0,0,2020-02-09
